In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
np.random.seed(42)

In [ ]:
# !unzip /content/drive/MyDrive/SAR/colorization/Input_Sorted-20250323T013151Z-001.zip -d /content/drive/MyDrive/SAR/colorization/

In [ ]:
# !unzip /content/drive/MyDrive/SAR/colorization/Output_Sorted-20250323T013202Z-001.zip -d /content/drive/MyDrive/SAR/colorization/

In [3]:
color_dir = '../data/colorization/color/train'
gray_dir = '../data/colorization/bw/train'

In [4]:
print(os.listdir(color_dir)[:5])
print(os.listdir(gray_dir)[:5])

['ROIs1868_summer_s2_59_p10.png', 'ROIs1868_summer_s2_59_p100.png', 'ROIs1868_summer_s2_59_p1000.png', 'ROIs1868_summer_s2_59_p1001.png', 'ROIs1868_summer_s2_59_p1002.png']
['ROIs1868_summer_s1_59_p10.png', 'ROIs1868_summer_s1_59_p100.png', 'ROIs1868_summer_s1_59_p1000.png', 'ROIs1868_summer_s1_59_p1001.png', 'ROIs1868_summer_s1_59_p1003.png']


In [7]:
SIZE = 256
HEIGHT = SIZE
WIDTH = SIZE
ImagePath = color_dir
N = 500

def ExtractInput(path):
    X_img = []
    y_img = []
    for imageDir in os.listdir(ImagePath)[:N]:
        try:
            img = cv2.imread(os.path.join(ImagePath, imageDir))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

            img = img.astype(np.float32)
            img_lab = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)
            #Convert the rgb values of the input image to the range of 0 to 1
            #1.0/255 indicates that we are using a 24-bit RGB color space.
            #It means that we are using numbers between 0–255 for each color channel
            #img_lab = 1.0/225*img_lab
            # resize the lightness channel to network input size
            img_lab_rs = cv2.resize(img_lab, (WIDTH, HEIGHT)) # resize image to network input size
            img_l = img_lab_rs[:,:,0] # pull out L channel
            #img_l -= 50 # subtract 50 for mean-centering
            img_ab = img_lab_rs[:,:,1:]#Extracting the ab channel
            img_ab = img_ab/128
            #The true color values range between -128 and 128. This is the default interval
            #in the Lab color space. By dividing them by 128, they too fall within the -1 to 1 interval.
            X_img.append(img_l)
            y_img.append(img_ab)
        except:
            pass
    X_img = np.array(X_img)
    y_img = np.array(y_img)

    return X_img,y_img

In [6]:
X_, y_ = ExtractInput(ImagePath)

In [7]:
from keras.layers import Conv2D,MaxPooling2D,UpSampling2D,Input,BatchNormalization,LeakyReLU,concatenate
from keras.models import Model

def InstantiateModel(in_):
    model_ = Conv2D(16,(3,3),padding='same',strides=1)(in_)
    model_ = LeakyReLU()(model_)
    #model_ = Conv2D(64,(3,3), activation='relu',strides=1)(model_)
    model_ = Conv2D(32,(3,3),padding='same',strides=1)(model_)
    model_ = LeakyReLU()(model_)
    model_ = BatchNormalization()(model_)
    model_ = MaxPooling2D(pool_size=(2,2),padding='same')(model_)

    model_ = Conv2D(64,(3,3),padding='same',strides=1)(model_)
    model_ = LeakyReLU()(model_)
    model_ = BatchNormalization()(model_)
    model_ = MaxPooling2D(pool_size=(2,2),padding='same')(model_)

    model_ = Conv2D(128,(3,3),padding='same',strides=1)(model_)
    model_ = LeakyReLU()(model_)
    model_ = BatchNormalization()(model_)

    model_ = Conv2D(256,(3,3),padding='same',strides=1)(model_)
    model_ = LeakyReLU()(model_)
    model_ = BatchNormalization()(model_)

    model_ = UpSampling2D((2, 2))(model_)
    model_ = Conv2D(128,(3,3),padding='same',strides=1)(model_)
    model_ = LeakyReLU()(model_)
    model_ = BatchNormalization()(model_)

    model_ = UpSampling2D((2, 2))(model_)
    model_ = Conv2D(64,(3,3), padding='same',strides=1)(model_)
    model_ = LeakyReLU()(model_)
    #model_ = BatchNormalization()(model_)

    concat_ = concatenate([model_, in_])

    model_ = Conv2D(64,(3,3), padding='same',strides=1)(concat_)
    model_ = LeakyReLU()(model_)
    model_ = BatchNormalization()(model_)

    model_ = Conv2D(32,(3,3),padding='same',strides=1)(model_)
    model_ = LeakyReLU()(model_)
    #model_ = BatchNormalization()(model_)

    model_ = Conv2D(2,(3,3), activation='tanh',padding='same',strides=1)(model_)

    return model_

In [8]:
Input_Sample = Input(shape=(HEIGHT, WIDTH,1))
Output_ = InstantiateModel(Input_Sample)
Model_Colourization = Model(inputs=Input_Sample, outputs=Output_)

In [9]:
from keras.optimizers import Adam

LEARNING_RATE = 0.001
Model_Colourization.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                            loss='mean_squared_error')
Model_Colourization.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │        160 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 256, 256,  │          0 │ conv2d[0][0]      │
│ (LeakyReLU)         │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │      4,640 │ leaky_re_lu[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_1       │ (None, 256, 256,  │          0 │ conv2d_1[0][0]    │
│ (LeakyReLU)         │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256, 256,  │        128 │ leaky_re_lu_1[0]… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_2       │ (None, 128, 128,  │          0 │ conv2d_2[0][0]    │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ leaky_re_lu_2[0]… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 64,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_3       │ (None, 64, 64,    │          0 │ conv2d_3[0][0]    │
│ (LeakyReLU)         │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        512 │ leaky_re_lu_3[0]… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │    295,168 │ batch_normalizat… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_4       │ (None, 64, 64,    │          0 │ conv2d_4[0][0]    │
│ (LeakyReLU)         │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │      1,024 │ leaky_re_lu_4[0]

 Total params: 820,386 (3.13 MB)

 Trainable params: 819,042 (3.12 MB)

 Non-trainable params: 1,344 (5.25 KB)

In [10]:
def GenerateInputs(X_,y_):
    for i in range(len(X_)):
        X_input = X_[i].reshape(1,SIZE,SIZE,1)
        y_input = y_[i].reshape(1,SIZE,SIZE,2)
        yield (X_input,y_input)

from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X_,y_, random_state=42)

Model_Colourization.fit(
    GenerateInputs(X_, y_),
    epochs=50,
    verbose=1,
    steps_per_epoch=38,
    validation_data=GenerateInputs(X_val, y_val),
    validation_steps=10
)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 37s 864ms/step - loss: 0.1113 - val_loss: 0.4062
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 32s 852ms/step - loss: 0.0189 - val_loss: 0.0882
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 33s 867ms/step - loss: 0.0048 - val_loss: 0.0472
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 34s 904ms/step - loss: 0.0023 - val_loss: 0.0175
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 33s 871ms/step - loss: 0.0019 - val_loss: 0.0045
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 34s 889ms/step - loss: 7.7506e-04 - val_loss: 0.0014
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 33s 873ms/step - loss: 5.4269e-04 - val_loss: 0.0012
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 34s 890ms/step - loss: 2.5161e-04 - val_loss: 7.1056e-04
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 33s 878ms/step - loss: 2.8527e-05 - val_loss: 4.1594e-04
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 34s 894ms/step - loss: 1.6963e-05 - val_loss: 3.7715e-04
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 33s 878ms/step - loss: 1.6606e-05 - val_loss: 1.4892e-04
E

c:\Users\Naman\Desktop\Repos\sar\model\env\Lib\site-packages\keras\src\trainers\epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


38/38 ━━━━━━━━━━━━━━━━━━━━ 8s 192ms/step - loss: 1.1241e-05 - val_loss: 1.2528e-04
Epoch 15/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - loss: 1.0980e-05 - val_loss: 2.1389e-04
Epoch 16/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 1.0930e-05 - val_loss: 2.1339e-04
Epoch 17/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 1.0886e-05 - val_loss: 2.1325e-04
Epoch 18/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 1.0841e-05 - val_loss: 2.1311e-04
Epoch 19/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - loss: 1.0795e-05 - val_loss: 2.1284e-04
Epoch 20/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 1.0748e-05 - val_loss: 2.1274e-04
Epoch 21/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 1.0704e-05 - val_loss: 2.1255e-04
Epoch 22/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - loss: 1.0661e-05 - val_loss: 2.1238e-04
Epoch 23/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - loss: 1.0621e-05 - val_loss: 2.1225e-04
Epoch 24/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - loss: 1.0577e-05 

In [11]:
Model_Colourization.save('../models/colorization-CNN.keras')

In [8]:
import tensorflow as tf
Model_Colourization = tf.keras.models.load_model('../models/colorization-CNN.keras')

In [9]:
X_test, y_test = ExtractInput('../data/colorization/color/test')

In [10]:
y_preds = Model_Colourization.predict(X_test)

16/16 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step


## PSNR

In [11]:
import numpy as np

def calculate_psnr(y_test, y_preds):
    mse = np.mean((y_test - y_preds) ** 2, axis=(1, 2, 3))  # Compute MSE per image
    max_pixel = 255.0  # Assuming 8-bit images
    psnr = 10 * np.log10((max_pixel ** 2) / (mse + 1e-10))  # Avoid division by zero
    return psnr

# Example usage:
psnr_values = calculate_psnr(y_test, y_preds)
average_psnr = np.mean(psnr_values)

print(f"Average PSNR: {average_psnr:.2f} dB")

Average PSNR: 91.59 dB


## SSIM

In [12]:
import numpy as np
from skimage.metrics import structural_similarity as ssim

def calculate_ssim(y_test, y_preds):
    ssim_values = []
    for i in range(y_test.shape[0]):
        ssim_index = ssim(
            y_test[i], y_preds[i], 
            data_range=255, 
            win_size=7,  # Explicitly setting window size
            channel_axis=-1  # Specifies the color channel axis
        )
        ssim_values.append(ssim_index)
    return np.array(ssim_values)

# Example usage:
ssim_values = calculate_ssim(y_test, y_preds)
average_ssim = np.mean(ssim_values)

print(f"Average SSIM: {average_ssim:.4f}")

Average SSIM: 1.0000


## ENL

In [13]:
import numpy as np

def calculate_enl(image):
    """
    Computes the Equivalent Number of Looks (ENL) for a given image.
    Assumes the image is a NumPy array in grayscale.
    """
    mean_intensity = np.mean(image)
    variance_intensity = np.var(image)

    if variance_intensity == 0:
        return float('inf')  # Avoid division by zero

    enl = (mean_intensity ** 2) / variance_intensity
    return enl

# Compute ENL for the ground truth and predicted images
enl_test = calculate_enl(y_test)
enl_preds = calculate_enl(y_preds)

print(f"ENL for Ground Truth (y_test): {enl_test:.2f}")
print(f"ENL for Predicted Image (y_preds): {enl_preds:.2f}")

ENL for Ground Truth (y_test): inf
ENL for Predicted Image (y_preds): 0.03


## MSE

In [14]:
import numpy as np

def calculate_mse(y_test, y_preds):
    """
    Compute the Mean Squared Error (MSE) between two images.
    Assumes both y_test and y_preds are NumPy arrays of the same shape.
    """
    mse = np.mean((y_test - y_preds) ** 2)
    return mse

# Compute MSE for the dataset
mse_value = calculate_mse(y_test, y_preds)

print(f"Mean Squared Error (MSE): {mse_value:.4f}")

Mean Squared Error (MSE): 0.0002


## LPIPS

In [16]:
import torch
import lpips
import numpy as np

lpips_model = lpips.LPIPS(net='alex')

def calculate_lpips(y_test, y_preds, lpips_model):
    """
    Compute LPIPS score between two image datasets.
    Assumes y_test and y_preds are NumPy arrays with shape (N, H, W, C) and pixel values in [0, 1].
    """

    # Ensure images have three channels (convert grayscale to RGB)
    if y_test.shape[-1] == 1:
        y_test = np.repeat(y_test, 3, axis=-1)
    if y_preds.shape[-1] == 1:
        y_preds = np.repeat(y_preds, 3, axis=-1)

    # Convert NumPy arrays to PyTorch tensors
    y_test_torch = torch.tensor(y_test).permute(0, 3, 1, 2).float()  # Change to (N, C, H, W)
    y_preds_torch = torch.tensor(y_preds).permute(0, 3, 1, 2).float()

    # Ensure tensor values are in [-1, 1] (required for LPIPS)
    y_test_torch = 2 * y_test_torch - 1
    y_preds_torch = 2 * y_preds_torch - 1

    # Debugging: Print tensor shapes
    print(f"y_test_torch shape: {y_test_torch.shape}")
    print(f"y_preds_torch shape: {y_preds_torch.shape}")

    # Compute LPIPS
    lpips_scores = lpips_model(y_test_torch, y_preds_torch)
    return lpips_scores.mean().item()

# Example test data (Ensure correct shape)
N, H, W = 10, 256, 256
y_test = np.random.rand(N, H, W, 3).astype(np.float32)  # Simulated RGB images
y_preds = np.random.rand(N, H, W, 3).astype(np.float32)  # Simulated RGB images

# Compute LPIPS for the dataset
lpips_value = calculate_lpips(y_test, y_preds, lpips_model)

print(f"LPIPS Score: {lpips_value:.4f}")


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: c:\Users\Naman\Desktop\Repos\sar\model\env\Lib\site-packages\lpips\weights\v0.1\alex.pth
y_test_torch shape: torch.Size([10, 3, 256, 256])
y_preds_torch shape: torch.Size([10, 3, 256, 256])
LPIPS Score: 0.1999


In [21]:
!pip install torch-fidelity

In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
from torchmetrics.image.fid import FrechetInceptionDistance

def calculate_fid(y_test, y_preds):
    """
    Compute FID score between two datasets.
    Assumes y_test and y_preds are NumPy arrays with shape (N, H, W, C) and pixel values in [0, 1].
    """

    # Convert images to tensors and normalize to [0, 255]
    y_test_torch = torch.tensor(y_test).permute(0, 3, 1, 2) * 255
    y_preds_torch = torch.tensor(y_preds).permute(0, 3, 1, 2) * 255

    # Ensure dtype is uint8 (required for FID)
    y_test_torch = y_test_torch.to(torch.uint8)
    y_preds_torch = y_preds_torch.to(torch.uint8)

    # Initialize FID metric
    fid = FrechetInceptionDistance(feature=2048)  # Uses InceptionV3 features

    # Compute FID score
    fid.update(y_test_torch, real=True)
    fid.update(y_preds_torch, real=False)
    fid_score = fid.compute().item()

    return fid_score

# Example test data (Ensure correct shape)
N, H, W = 10, 256, 256
y_test = np.random.rand(N, H, W, 3).astype(np.float32)  # Simulated real RGB images
y_preds = np.random.rand(N, H, W, 3).astype(np.float32)  # Simulated generated RGB images

# Compute FID
fid_value = calculate_fid(y_test, y_preds)
print(f"FID Score: {fid_value:.4f}")

Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to C:\Users\Naman/.cache\torch\hub\checkpoints\weights-inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:11<00:00, 8.07MB/s]


FID Score: 39.9818


In [2]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
from torchmetrics.image.inception import InceptionScore

def calculate_inception_score(y_preds):
    """
    Compute Inception Score (IS) for generated images.
    Assumes y_preds are NumPy arrays with shape (N, H, W, C) and pixel values in [0, 1].
    """

    # Convert images to tensors and normalize to [0, 255]
    y_preds_torch = torch.tensor(y_preds).permute(0, 3, 1, 2) * 255

    # Ensure dtype is uint8 (required for IS)
    y_preds_torch = y_preds_torch.to(torch.uint8)

    # Initialize Inception Score metric
    inception_score = InceptionScore(feature=2048)

    # Compute IS
    inception_score.update(y_preds_torch)
    is_mean, is_std = inception_score.compute()

    return is_mean.item(), is_std.item()

# Example generated data (Ensure correct shape)
N, H, W = 10, 256, 256
y_preds = np.random.rand(N, H, W, 3).astype(np.float32)  # Simulated generated RGB images

# Compute IS
is_mean, is_std = calculate_inception_score(y_preds)
print(f"Inception Score: {is_mean:.4f} ± {is_std:.4f}")

c:\Users\Naman\Desktop\Repos\sar\model\env\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Inception Score: 1.0000 ± 0.0000
